### Laboratorium Big Data
<i> Antoni Perużyński </i>

**Zaliczenie części Wprowadzenie do Sparka**

Zbior danych to oceny filmow. Dane pochodza z projektu https://movielens.org/
Wiecej informacji o danych mozna znalezc pod adresem: http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

In [40]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame_add').master('local[*]').getOrCreate()

import pyspark.sql.functions as f

In [60]:
import matplotlib.pyplot as plt

In [3]:
data_path = 'ml-latest-small/'
movies = spark.read.option("header", "true").csv(data_path+'movies.csv')
ratings = spark.read.option("header", "true").csv(data_path+'ratings.csv')

Do rozważenia: Zalecanym formatem przechowywania danych jest Apache Parquet.
Wiecej o tym formacie danych mozna sie dowiedziec np. tutaj: https://parquet.apache.org/.
W skrocie, tak strona projektu podsumowuje misje stojaca za tym formatem:
"We created Parquet to make the advantages of compressed, efficient columnar data representation available to any project in the Hadoop ecosystem".

Przyjrzyj sie danym i przeanalizuj. 

Sugestie:
1. Z jakich kolumn skladaja sie wczytane powyzej zbiory?
2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
4. Czy w zbiorze znajduja sie braki danych?
5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?
6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
7. Jaki procent filmow ma tylko maksymalne oceny?
8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
9. Jaki jest rozklad ocen?
10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?
11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?
12. Jakie sa roznice pomiedzy liczba ocen w zbiorze z roku na rok? Zaloz, ze timestamp reprezentuje liczbe sekund od roku 1960.
13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

### 1. Z jakich kolumn skladaja sie wczytane powyzej zbiory?

In [5]:
movies.columns

['movieId', 'title', 'genres']

In [6]:
ratings.columns

['userId', 'movieId', 'rating', 'timestamp']

### 2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?

In [8]:
movies.groupBy().count().show()

+-----+
|count|
+-----+
| 9742|
+-----+



In [9]:
movies.groupBy().count().show()

+-----+
|count|
+-----+
| 9742|
+-----+



### 3. Oceny ilu uzytkownikow znajduja sie w zbiorze?

In [15]:
df = ratings.groupBy('userId').count()
df.count()

610

### 4. Czy w zbiorze znajduja sie braki danych?

In [22]:
ratings.filter(ratings.userId.isNull() | ratings.movieId.isNull() | ratings.rating.isNull() | ratings.timestamp.isNull()).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
+------+-------+------+---------+



In [23]:
movies.filter(movies.title.isNull() | movies.movieId.isNull() | movies.genres.isNull()).show()

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
+-------+-----+------+



### 5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?

In [37]:
dftogether = movies.join(other=ratings, on='movieId', how='left')
dftogether.filter(dftogether.rating.isNull()).count()

18

In [34]:
dftogether.filter(dftogether.rating.isNull()).show()

+-------+--------------------+--------------------+------+------+---------+
|movieId|               title|              genres|userId|rating|timestamp|
+-------+--------------------+--------------------+------+------+---------+
|   1076|Innocents, The (1...|Drama|Horror|Thri...|  null|  null|     null|
|   2939|      Niagara (1953)|      Drama|Thriller|  null|  null|     null|
|   3338|For All Mankind (...|         Documentary|  null|  null|     null|
|   3456|Color of Paradise...|               Drama|  null|  null|     null|
|   4194|I Know Where I'm ...|   Drama|Romance|War|  null|  null|     null|
|   5721|  Chosen, The (1981)|               Drama|  null|  null|     null|
|   6668|Road Home, The (W...|       Drama|Romance|  null|  null|     null|
|   6849|      Scrooge (1970)|Drama|Fantasy|Mus...|  null|  null|     null|
|   7020|        Proof (1991)|Comedy|Drama|Romance|  null|  null|     null|
|   7792|Parallax View, Th...|            Thriller|  null|  null|     null|
|   8765|Thi

### 6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

In [51]:
df6 = dftogether.groupBy('title').agg(f.mean('rating').alias("mean_rating"),\
                                     f.count('title').alias("votes_number")).orderBy(f.desc('mean_rating'), f.desc('votes_number')).show()


+--------------------+-----------+------------+
|               title|mean_rating|votes_number|
+--------------------+-----------+------------+
|Enter the Void (2...|        5.0|           2|
| Lesson Faust (1994)|        5.0|           2|
|Jonah Who Will Be...|        5.0|           2|
| Belle époque (1992)|        5.0|           2|
|Heidi Fleiss: Hol...|        5.0|           2|
|     Lamerica (1994)|        5.0|           2|
|Come and See (Idi...|        5.0|           2|
|George Carlin: Yo...|        5.0|           1|
|Vacations in Pros...|        5.0|           1|
|Tickling Giants (...|        5.0|           1|
|English Vinglish ...|        5.0|           1|
|      Villain (1971)|        5.0|           1|
|Winnie the Pooh a...|        5.0|           1|
|Paper Birds (Pája...|        5.0|           1|
|Awfully Big Adven...|        5.0|           1|
|         Rain (2001)|        5.0|           1|
|    Radio Day (2008)|        5.0|           1|
|National Lampoon'...|        5.0|      

### 7. Jaki procent filmow ma tylko maksymalne oceny?

In [57]:
df7 = dftogether.groupBy('title').agg(f.min('rating').alias("min_rating"))
df7Only5 = df7.where(df7.min_rating ==5)
df7Only5.count()/movies.count()*100

3.03839047423527

### 8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

In [58]:
df6 = dftogether.groupBy('title').agg(f.min('rating').alias("min_rating"),\
                                     f.count('title').alias("votes_number")).orderBy(f.desc('min_rating'), f.desc('votes_number')).show()

+--------------------+----------+------------+
|               title|min_rating|votes_number|
+--------------------+----------+------------+
|     Lamerica (1994)|       5.0|           2|
|Enter the Void (2...|       5.0|           2|
| Lesson Faust (1994)|       5.0|           2|
|Heidi Fleiss: Hol...|       5.0|           2|
| Belle époque (1992)|       5.0|           2|
|Come and See (Idi...|       5.0|           2|
|Jonah Who Will Be...|       5.0|           2|
|Adventures Of She...|       5.0|           1|
|Alien Contaminati...|       5.0|           1|
|7 Faces of Dr. La...|       5.0|           1|
| 'Salem's Lot (2004)|       5.0|           1|
|Battle Royale 2: ...|       5.0|           1|
|All the Vermeers ...|       5.0|           1|
|    12 Chairs (1976)|       5.0|           1|
|Animals are Beaut...|       5.0|           1|
|A Detective Story...|       5.0|           1|
|A Perfect Day (2015)|       5.0|           1|
|    All Yours (2016)|       5.0|           1|
|American Fri

### 9. Jaki jest rozklad ocen?

In [70]:
ratings.groupBy("rating").count().orderBy("rating").show()

+------+-----+
|rating|count|
+------+-----+
|   0.5| 1370|
|   1.0| 2811|
|   1.5| 1791|
|   2.0| 7551|
|   2.5| 5550|
|   3.0|20047|
|   3.5|13136|
|   4.0|26818|
|   4.5| 8551|
|   5.0|13211|
+------+-----+



###  10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?

In [78]:
movies.where(movies.genres.like('%Documentary%')).count()

440

### 11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?

In [86]:
dfdocumentary = dftogether.where(dftogether.genres.like('%Documentary%'))
dfdocumentary2 = dfdocumentary.groupBy('title').agg(f.mean('rating').alias("mean_rating"),\
                                     f.count('title').alias("votes_number")).orderBy(f.desc('mean_rating'), f.desc('votes_number'))
dfdocumentary2.where(dfdocumentary2['votes_number']>=10).show(1)

+--------------------+------------------+------------+
|               title|       mean_rating|votes_number|
+--------------------+------------------+------------+
|Fog of War: Eleve...|4.3076923076923075|          13|
+--------------------+------------------+------------+
only showing top 1 row



### 12. Jakie sa roznice pomiedzy liczba ocen w zbiorze z roku na rok? Zaloz, ze timestamp reprezentuje liczbe sekund od roku 1960.

In [103]:
df12 = ratings.withColumn(colName = 'Year', col = f.floor(1960 + ratings.timestamp / (60*60*24*365.2425)))
df12.groupBy('Year').count().orderBy("Year").show()

+----+-----+
|Year|count|
+----+-----+
|1986| 6031|
|1987| 1925|
|1988|  507|
|1989| 2439|
|1990|10061|
|1991| 3922|
|1992| 3478|
|1993| 4014|
|1994| 3274|
|1995| 5818|
|1996| 4059|
|1997| 7114|
|1998| 4345|
|1999| 4163|
|2000| 2302|
|2001| 1690|
|2002| 4656|
|2003| 1664|
|2004| 1439|
|2005| 6616|
+----+-----+
only showing top 20 rows



### 13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

In [126]:
df12 = movies.withColumn(colName = 'genresNumber', col = movies.genres.contains("|"))
df12.show()

+-------+--------------------+--------------------+------------+
|movieId|               title|              genres|genresNumber|
+-------+--------------------+--------------------+------------+
|      1|    Toy Story (1995)|Adventure|Animati...|        true|
|      2|      Jumanji (1995)|Adventure|Childre...|        true|
|      3|Grumpier Old Men ...|      Comedy|Romance|        true|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|        true|
|      5|Father of the Bri...|              Comedy|       false|
|      6|         Heat (1995)|Action|Crime|Thri...|        true|
|      7|      Sabrina (1995)|      Comedy|Romance|        true|
|      8| Tom and Huck (1995)|  Adventure|Children|        true|
|      9| Sudden Death (1995)|              Action|       false|
|     10|    GoldenEye (1995)|Action|Adventure|...|        true|
|     11|American Presiden...|Comedy|Drama|Romance|        true|
|     12|Dracula: Dead and...|       Comedy|Horror|        true|
|     13|        Balto (1

In [125]:
df13= movies.withColumn('genresNumber',f.length('genres')) - f.length(f.regexp_replace('genres', '|', '')).show()

TypeError: 'Column' object is not callable